# Experiment Notebook: System Metrics

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: Sanity Checks](#Analysis-1:-Sanity-Checks)
* [Analysis 2: Correlation Matrix](#Analysis-1:-Correlation-Matrix)
* [Analysis 3: PCV at Risk](#Analysis-3:-PCV-at-Risk)
* [Analysis 4: Capital Allocation Metrics](#Analysis-4:-Capital-Allocation-Metrics)
* [Assorted Metrics](#Analysis-4:-Assorted-Metrics)

# Experiment Summary 

The purpose of this notebook is to demonstrate the system's standard metrics, KPIs and goals.

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

In [ ]:
# Inspect experiment template
display_code(default_experiment)

In [ ]:
# Create a simulation for each analysis
simulation_1 = copy.deepcopy(default_experiment.experiment.simulations[0])
simulation_2 = copy.deepcopy(default_experiment.experiment.simulations[0])
simulation_3 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
# Experiment configuration
# simulation_1.model.initial_state.update({})
# simulation_1.model.params.update({})

# Risk Analysis Methodology

The goal of the FEI Ecosystem Model Risk Analysis is to provide qualitative and quantitative recommendations to surface the most appropriate FEI monetary policy parameter settings across multiple scenarios. The risk analysis includes a cohesive set of model metrics, risk scores and protocol KPIs.

The main holistic risk analysis and parameter recommendation tool at our disposal is the FEI Capital Allocation model, where we allow the model to provide risk-weighted target allocations of FEI in all modeled avenues for FEI liquidity - LP, MM, FSD, Idle. These allocation targets will depend on parameter sweeps of FEI's monetary policy levers such as the FEI Savings Rate (see simplified ERD). The highest scoring parameter settings in terms of FEI Capital allocation and associated comparative KPIs will form the quantitative basis for recommendations.

It is to be noted that we can analyze KPIs comparatively based on monetary policy parameter settings in <b>Two ways</b> - qualitative/deterministic, and statistically based/stochastic.

Since the main driver of volatility in the FEI ecosystem is the Volatile Asset (an abstraction for Ethereum), we can model this volatile asset as Trajectory-based (based on a linear function), or based on a stochastic process such as a Geometric Brownian motion. The application of both settings for the Volatile asset allows us to respectively perform two analysis types.

1. Qualitative recommendations can be made as a result of comparative KPI analysis across different monetary policy parameter settings, using a trajectory model for the Volatile Asset. These do NOT involve monte carlo runs. The output of these analyses is to understand the impact on KPIs as a result of monetary policy changes for various final levels of Volatile Asset price. Ex: For a 30% VA price downturn, Stable backing ratio is higher with monetary policy 1 than policy 2, ie: Delta_1,2 Stable Backing ratio > 0, hence policy 1 is recommended.


2. Statistically-based recommendations can be made as a result of comparative KPI analysis across different monetary policy parameter settings, using a parameterized stochastic model for the Volatile Asset. These DO involve monte carlo runs. The output of these analyses is to construct a probability distribution for each KPI from which summary statistics can be derived. This allows us to empirically say that for a given parameter setting, KPIs are above or below key thresholds with a certain probability. Ex: Over 1000 simulations, 1-Day PCV at Risk is < 1M USD with a 90% probability with policy 1 and < 1M USD with an 85% probability with policy 2. Additionally, the statistical average PCVaR is lower with policy 1 than with policy 2, ie: Delta_1,2 avg. PCVaR > 0. Hence policy 1 is recommended.


# Analysis 1: PCV Sanity Checks

A simulation across 4 volatile asset price scenarios to illustrate PCV state evolution. Here, deterministic price trajectories for the Volatile Asset price are used, as opposed to parameterized stochastic processes.

In [ ]:
# Analysis-specific setup
simulation_1.model.params.update({
    "volatile_asset_price_process": [
        lambda _run, _timestep: 2_000,
        lambda _run, timestep: 2_000 if timestep < 365 / 4 else (1_000 if timestep < 365 * 3/4 else 2_000),
        lambda _run, timestep: 2_000 * (1 + timestep * 0.2 / 365),
        lambda _run, timestep: 2_000 * (1 - timestep * 0.2 / 365),
    ],
})

In [ ]:
# Experiment execution
df, exceptions = run(simulation_1)

In [ ]:
# Post-processing and visualizations

In [ ]:
fig = df.plot(y='volatile_asset_price', color='subset')

fig.update_xaxes(title='Timestamp')

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import itertools
from experiments.notebooks.visualizations.plotly_theme import cadlabs_colorway_sequence
color_cycle = itertools.cycle(cadlabs_colorway_sequence)


fig = make_subplots(rows=5, cols=len(df.subset.unique()), shared_yaxes=True)

for subset in df.subset.unique():
    df_plot = df.query('subset == @subset')
    
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.volatile_asset_price,
            name="Volatile Asset Price",
            line=dict(color=cadlabs_colorway_sequence[0]),
            showlegend=(True if subset == 0 else False),
        ),
        row=1, col=subset+1,
    )

    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.total_pcv,
            name="Total PCV",
            line=dict(color=cadlabs_colorway_sequence[1]),
            showlegend=(True if subset == 0 else False),
        ),
        row=2, col=subset+1
    )

    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.collateralization_ratio,
            name="Collateralization Ratio",
            line=dict(color=cadlabs_colorway_sequence[2]),
            showlegend=(True if subset == 0 else False),
        ),
        row=3, col=subset+1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.total_volatile_asset_pcv,
            name="Volatile Asset PCV",
            line=dict(color=cadlabs_colorway_sequence[3]),
            showlegend=(True if subset == 0 else False),
            stackgroup='one',
        ),
        row=4, col=subset+1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.total_stable_asset_pcv,
            name="Stable Asset PCV",
            line=dict(color=cadlabs_colorway_sequence[4]),
            showlegend=(True if subset == 0 else False),
            stackgroup='one',
        ),
        row=4, col=subset+1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.liquidity_pool_tvl,
            name="Liquidity Pool TVL",
            line=dict(color=cadlabs_colorway_sequence[4]),
            showlegend=(True if subset == 0 else False),
        ),
        row=5, col=subset+1
    )


fig.update_layout(height=1000, title_text="PCV Sanity Checks")
fig.show()

# Analysis 2: Correlation Matrix

In [ ]:
# Analysis-specific setup
simulation_2.model.params.update({})

In [ ]:
# Experiment execution
df_2, exceptions = run(simulation_2)

In [ ]:
# Post-processing and visualizations

In [ ]:
import plotly.graph_objects as go


variables = [
    "volatile_asset_price",
    "total_pcv",
    "collateralization_ratio",
    "total_stable_asset_pcv",
    "liquidity_pool_tvl"
]

z = df_2[variables].corr().values.tolist()

fig = px.imshow(z, x=variables, y=variables, color_continuous_scale='RdBu_r', origin='lower')
fig.update_xaxes(side="top")
fig.show()

fig = go.Figure(data=go.Heatmap(
        z=z,
        x=variables,
        y=variables,
        colorscale='Inferno'
    )
)

fig.show()

# Analysis 3: PCV at Risk

In [ ]:
from operator import lt, gt


simulation_3.runs = 10

parameter_overrides = {
    "target_rebalancing_condition": [gt, lt], # Simulate decrease and increase of stable PCV
    "target_stable_pcv_ratio": [0.2, 0.5], # Simulate decrease and increase of stable PCV
    "rebalancing_period": [int(365 / 4)],
}

simulation_3.model.params.update(parameter_overrides)

In [ ]:
# Experiment execution
df_3, exceptions = run(simulation_3)

In [ ]:
df = df_3

In [ ]:
df.plot(y="volatile_asset_price", color="run", facet_row="subset")

In [ ]:
df.plot(y="total_pcv", color="run", facet_row="subset")

## Vectorized PCV at Risk

In this analysis, which is based on a stochastic Volatile Asset Price process with 10 realizations (10 monte carlo runs), we look at two policies (a parameter sweep of size 2) and look at the empirical probability of the PCV at Risk KPI being below a certain threshold, as well as comparatively examine the value of the KPI across both policies to yield a recommendation.

In [ ]:
def calculate_VaR_run(df, n_run, alpha, n_timesteps, t_start, t_end):
    pcv_ret = df.query('run==@n_run and (timestep > @t_start and timestep <= @t_end)')['total_pcv'].pct_change()
    pcv_final_val = df.query('run==@n_run')['total_pcv'].iloc[-1]
    q = pcv_ret.quantile(1-alpha)
    # see https://www0.gsb.columbia.edu/faculty/pglasserman/B6014/var-d.pdf
    # for n-day var simplifying assumption which allows for generalization
    VaR_n = abs(pcv_final_val * q)*np.sqrt(n_timesteps)
    
    return VaR_n, q

In [ ]:
def calculate_VaR_subset(df, n_subset, alpha, n_timesteps, t_start, t_end):
    VAR = []
    
    df_ = df.query("subset==@n_subset")
    for run in df_['run'].value_counts().index:
        var, q = calculate_VaR_run(df_, run, alpha, n_timesteps, t_start, t_end)
        
        VAR.append((n_subset, var, q))
    
    return pd.DataFrame(VAR, columns=[x+'_'+str(n_timesteps) for x in ['subset', 'VaR', 'q']])

In [ ]:
def calculate_VaR(df, alpha, n_timesteps, t_start, t_end):
    L = []
    
    for subset in df['subset'].value_counts().index:
        VaR_subset = calculate_VaR_subset(df, subset, alpha, n_timesteps, t_start, t_end)
        L.append(VaR_subset)
        
    return pd.concat(L, axis=0).reset_index(drop=True)

In [ ]:
def calculate_VaR_n(df, alpha, timestep_range, t_start, t_end):
    U, L = [], []
    
    for t in range(timestep_range):
        L.append(calculate_VaR(df, 0.95, t+1, t_start, t_end))
        U.append(t+1)
        
    return dict(zip(U, L))

In [ ]:
def calculate_VaR_summary_stats(df, n_timesteps):
    L = []
    colnames = []
    for subset in df['subset'+'_'+str(n_timesteps)].value_counts().index:
        L.append(df.query('subset'+'_'+str(n_timesteps)+'==@subset').describe())
        colnames += [colname+'_'+str(subset) for colname in df.columns]
    
    VAR_info = pd.concat(L, axis=1)
    VAR_info.columns = colnames
    VAR_info = VAR_info.drop(index=['count'])
    return VAR_info
        

In [ ]:
# calculate 1-day vectorized VaR for all simulation outputs
# set window bounds, whole simulation for simplicity
t_start = 0
t_end = 365
alpha = 0.95
max_day_VaR = 10
VAR_df_dict = calculate_VaR_n(df, alpha, max_day_VaR, t_start, t_end)

# NOTE: create rolling window by further vectorizing VaR calculation by iterating over start and end time

In [ ]:
VAR_1_stats = calculate_VaR_summary_stats(VAR_df_dict[1], 1)

In [ ]:
VAR_10_stats = calculate_VaR_summary_stats(VAR_df_dict[10], 10)

## 1-day PCV at Risk

A visualization of PCVaR calculation for one monte carlo run for one Policy (parameter setting).

In [ ]:
alpha = 0.95
pcv_ret = df.query('run==1 and subset==0')['total_pcv'].pct_change()
var, q = calculate_VaR_run(df, 1, alpha, 1, 0, 365)


fig = px.histogram(pcv_ret, x="total_pcv", title="Histogram of PCV Returns for Run 1, Policy 0")
fig.add_vline(x=q)

fig.show()
print('1-Day PCVar for Run 1, Policy 0 (Subset 0) is', np.round(var,2),
      'USD with 5% quantile value', 100*np.round(q, 4),'%')


PCVaR Summary Statistics across all monte carlo runs for each policy (parameter subset)

In [ ]:
VAR_1_stats

In [ ]:
print(f"1-day average PCV at Risk at 95th quantile for subset 0: \n {np.round(VAR_1_stats['VaR_1_0'].loc['mean'], 2):,} USD")

In [ ]:
print(f"1-day average PCV at Risk at 95th quantile for subset 1: \n {np.round(VAR_1_stats['VaR_1_1'].loc['mean'], 2):,} USD")

### Empirical Probability of KPI at Threshold analyisis

In [ ]:
def calculate_VaR_threshold_probability(df, n_timesteps, threshold):
    L = []
    colnames = []
    for subset in df['subset'+'_'+str(n_timesteps)].value_counts().index:
        df_ = (df.query('subset'+'_'+str(n_timesteps)+'==@subset')['q_'+str(n_timesteps)] <= threshold).astype(int)
        emp_probability = df_.sum()/len(df_)
        L.append(emp_probability)
        colnames.append('q_'+str(n_timesteps)+'_'+str(subset))
    return dict(zip(colnames,L))

In [ ]:
# check for how many simulation runs PCV at Risk is < 1% of total PCV
quantile_return_threshold = -1e-2
q_probabilities = calculate_VaR_threshold_probability(VAR_df_dict[1], 1, Quantile_return_threshold)

In [ ]:
print('For Policy 1, the 1-Day PCV at risk is less than 1% with a', 100*q_probabilities['q_1_0'],'% probability')
print('For Policy 2, the 1-Day PCV at risk is less than 1% with a', 100*q_probabilities['q_1_1'],'% probability')

From this analysis, we see that over the 10 monte carlo runs for each policy (each subset), since the probability of PCV at risk being less than 1% of total PCV on any given day is lower for policy 1 than for policy 2, policy 1 does a better job at risk mitigation, hence we recommend policy 1.

### Comparative Average KPI analysis

In [ ]:
avg_VaR_delta = np.round(VAR_1_stats['VaR_1_0'].loc['mean'], 2) - np.round(VAR_1_stats['VaR_1_1'].loc['mean'], 2)

In [ ]:
avg_VaR_quantile_delta = np.round(VAR_1_stats['q_1_0'].loc['mean'] - VAR_1_stats['q_1_1'].loc['mean'], 4)

In [ ]:
print(f"The Average PCVaR Delta between parameter for policies 1 and 2 is: \n {avg_VaR_delta:,} USD")

In [ ]:
print(f"The Average PCVaR Quantile Delta between parameter for policies 1 and 2 is: \n {avg_VaR_quantile_delta:,}")

We conclude that while the 1-Day PCVaR is greater for policy 1 than for policy 2, meaning more PCV is at risk on any given day at a 95% quantile, the value of this quantile is lower, meaning the PCV has a lower magnitude of negative returns, attesting to the risk mitigating effect of policy 1. Hence, policy 1 is recommended.

## 10-day PCV at Risk

In [ ]:
VAR_10_stats

In [ ]:
# print('Over the simulation, for parameters in the sweep corresponding to subset 0,',
#       np.round(VAR_10_stats['VaR_10_0'].loc['mean'], 2), 'USD is the mean 10-Day PCVaR at 95% over the 20 runs performed, with associated',
#       np.round(VAR_10_stats['q_10_0'].loc['mean'], 4), '% average 5% quantile percentile loss')

In [ ]:
# print('Over the simulation, for parameters in the sweep corresponding to subset 1,',
#       np.round(VAR_10_stats['VaR_10_1'].loc['mean'], 2), 'USD is the mean 10-Day PCVaR at 95% over the 20 runs performed, with associated',
#       np.round(VAR_10_stats['q_10_1'].loc['mean'], 4), '% average 5% quantile percentile loss')

In [ ]:
print(f"10-day PCV at Risk at 95th quantile for subset 0: \n {np.round(VAR_10_stats['VaR_10_0'].loc['mean'], 2):,} USD")

In [ ]:
print(f"10-day PCV at Risk at 95th quantile for subset 1: \n {np.round(VAR_10_stats['VaR_10_1'].loc['mean'], 2):,} USD")

## Summary Statistics

In [ ]:
_df = df.groupby(['subset','timestep']).mean().query('subset == 0')
stats_df = _df.describe()
stats_df.loc['skew'] = _df.skew()
stats_df.loc['kurtosis'] = _df.kurtosis()
# TODO: max drawdown & other relevant summary stats here

stats_df

# Analysis 4: Capital Allocation Metrics

In [ ]:
fei_capital_allocation_variables = [
    'fei_deposit_idle_balance',
    'fei_deposit_liquidity_pool_balance',
    'fei_deposit_money_market_balance'
]
fei_capital_allocation_variables.sort()

In [ ]:
import plotly.express as px

px.area(df_2.sort_index(), y=fei_capital_allocation_variables, groupnorm='percent')

In [ ]:
df_allocations = df[fei_capital_allocation_variables].iloc[-1]

px.pie(df_allocations.sort_index(), title='FEI Capital Allocation', values=df_allocations.values, names=df_allocations.index)

# Assorted Metrics

In [ ]:
df_2.plot(y="collateralization_ratio")

In [ ]:
df_2.plot(y="stable_backing_ratio")

In [ ]:
df_2.plot(y="stable_pcv_ratio")

In [ ]:
df_2.plot(y="pcv_yield_rate")

In [ ]:
df_2["pcv_yield_ratio"] = df_2["pcv_yield"] / df_2["total_user_circulating_fei"] * 365 / df_2["dt"]

df_2.plot(y="pcv_yield_ratio")